In [82]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os
import platform
import shutil
import networkx as nx


In [83]:
# target river segment that is downstream of lake victoria
target_ID = 2062605


# read the network topology that is HDMA hydrolakes global
ds = xr.open_dataset('/Users/shg096/Desktop/mizuRoute_HDMA_Global/data/Network_topology_HDMA_HydroLake_v1.nc')

# convert ds back to df
df = ds.to_dataframe()

# make the string out of ID and then slice the dataframe based on 2 (africa) and 7 lakes
df ['ID_str'] = df['ID'].astype(str)
df = df[df['ID_str'].astype(str).str.startswith(('2', '7'))]

# create the riv graph
riv_graph = nx.from_pandas_edgelist(df,\
                                    source='ID',\
                                    target='ID_next',\
                                    create_using=nx.DiGraph)
nodes = nx.ancestors(riv_graph, target_ID)
nodes = np.array(list(nodes))
nodes = np.append(nodes, target_ID)


# now slice the ds based on existing nodes
idx = np.where(np.isin(ds['ID'].values, nodes))
idx = np.array(idx).flatten()
ds_slice = ds.isel(n=idx)

# find the location of target segment and add -9999 as downstream
idx = np.where(np.isin(ds_slice['ID'].values, target_ID))
idx = np.array(idx).flatten()
ds_slice ['ID_next'][idx] = -9999


# change lake victoria values for target volume values
idx = np.where(np.isin(ds_slice['ID'].values, 7000016))
idx = np.array(idx).flatten()
ds_slice ['lake_type'][idx] = -1 # not parameteric lake type is assigned to lake victoria
ds_slice ['target_vol'] = ds_slice ['lake_type'] # initializing the 
ds_slice ['target_vol'][:] = ds_slice ['lake_type'][:] # initializing the 
ds_slice ['target_vol'][idx] = 1

   

[2011137 2021382 2004999 2054152 2054162 2007063 2019355 2017308 2025500
 2045986 2056231 2011178 2011182 2002994 2009151 2062409 2009164 2019412
 2025557 2037845 2017367 2015321 2041954 2033762 2015348 2017402 2052220
 2031741 2025598 2050176 2037890 2017420 2013325 2033808 2023573 2056341
 2046103 2001048 2017434 2060443 2033824 2029731 2021541 2062506 2001067
 2050221 2044078 2029752 2019512 2023610 2013373 2033853 2033855 2011331
 2013382 2048199 2017483 2046158 2013398 2062556 2003167 2062561 2062562
 2046178 2011364 2015469 2009332 2009333 2029818 2048254 2054404 2040069
 2040073 2056458 2031886 2001166 2025743 2031892 2033941 2042141 2033949
 2009376 2021674 2060588 2036024 2027836 2013508 2054473 2054476 2009427
 2036053 2054502 2001257 2003307 2048368 2031988 2050434 2046339 2009478
 2042246 2013577 2013582 2044304 2032020 2056597 2056596 2048407 2044312
 2019738 2027938 2017700 2005418 2040239 2056627 2060729 2032061 2015683
 2040259 2044371 2009566 2005473 2030052 2054633 20

In [86]:
# save the ds_slice
#ds_slice.to_netcdf('../data/Network_topology_lake_victoria.nc')

var_info = {'ID':        {'long_name': 'ID',        'unit': '-'},
            'ID_next':   {'long_name': 'ID next',   'unit': '-'},
            'length':    {'long_name': 'length',    'unit': 'm'},
            'area':      {'long_name': 'area',      'unit': 'm**2'},
            'lake':      {'long_name': 'lake',      'unit': '-'},
            'endorheic': {'long_name': 'endorheic', 'unit': '-'},
            'S_0':       {'long_name': 'S_0',       'unit': 'm**3'},
            'S_max':     {'long_name': 'S_max',     'unit': 'm**3'},
            'Coeff':     {'long_name': 'Coeff',     'unit': 'd**-1'},
            'power':     {'long_name': 'power',     'unit': '-'},
            'lake_type': {'long_name': 'lake_type', 'unit': '-'},
            'slope':     {'long_name': 'slope',     'unit': 'm m**-1'},
            'lake_type_for_target_vol':
                         {'long_name': 'lake type for target vol lake Victoria',     'unit': '-'},
            'target_vol':{'long_name': 'target_vol','unit': '-'}}

for key in var_info.keys():
    for key1 in var_info[key].keys():
        print(var_info[key][key1])
        ds[key].attrs[key1] = var_info[key][key1]
        print(ds[key].attrs[key1])


if not os.path.isdir('../data'):
    os.makedirs('../data')
        
if os.path.isfile('../data/Network_topology_lake_victoria.nc'):
    os.remove('../data/Network_topology_lake_victoria.nc')


ds_slice.to_netcdf('../data/Network_topology_lake_victoria.nc',\
                   encoding = {var: {'_FillValue': -9999, 'zlib': True, 'complevel': 9} for var in ds.data_vars})

ds_slice = xr.open_dataset('../data/Network_topology_lake_victoria.nc')

ds_slice

ID
ID
-
-
ID next
ID next
-
-
length
length
m
m
area
area
m**2
m**2
lake
lake
-
-
endorheic
endorheic
-
-
S_0
S_0
m**3
m**3
S_max
S_max
m**3
m**3
Coeff
Coeff
d**-1
d**-1
power
power
-
-
lake_type
lake_type
-
-
slope
slope
m m**-1
m m**-1
lake type for target vol lake Victoria
lake type for target vol lake Victoria
-
-
target_vol
target_vol
-
-


<xarray.Dataset>
Dimensions:                   (n: 518)
Coordinates:
    ID                        (n) int64 ...
Dimensions without coordinates: n
Data variables: (12/13)
    ID_next                   (n) float64 ...
    length                    (n) float64 ...
    area                      (n) float64 ...
    lake                      (n) float64 ...
    endorheic                 (n) float64 ...
    S_0                       (n) float64 ...
    ...                        ...
    Coeff                     (n) float64 ...
    power                     (n) float64 ...
    lake_type                 (n) float64 ...
    slope                     (n) float64 ...
    target_vol                (n) float64 ...
    lake_type_for_target_vol  (n) float64 ...
Attributes:
    Conventions:  CF-1.6
    License:      The data were written by Shervan Gharari. They are under GPL.
    history:      Created Mon Jul  3 21:31:18 2023
    source:       Written by test script of utilities (https://github.com/She...